# 🧭 Dashboard for SAIL 2025 – Short-Term Crowd Predictions

This notebook prepares the code and structure for a **Streamlit dashboard** to visualize short-term crowd density forecasts for SAIL 2025.

## 📝 1. Introductie

In dit notebook bouwen we een **Streamlit-dashboard** om korte-termijn crowd density-voorspellingen 
voor SAIL 2025 te visualiseren.

Het dashboard:
- Laadt sensordata of demo-data.
- Gebruikt simpele voorspellingsmodellen (Persistence, Moving Average, Random Forest).
- Visualiseert resultaten op een interactieve kaart en tijdreeks.
- Genereert alerts wanneer crowd density boven een drempel komt.

We gebruiken:
- **Pandas, NumPy** – data handling  
- **Plotly** – visualisatie  
- **Streamlit** – dashboarding  
- **scikit-learn** – eenvoudige ML

## ⚙️ 2. Imports & Setup

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor

print("Pandas:", pd.__version__)

## 📁 3. Data loading functies

Hier definiëren we functies om timeseries-data en zones (GeoJSON) te laden.
Als de bestanden niet bestaan, wordt automatisch demo-data gegenereerd.

In [ ]:
def load_timeseries(path: str) -> pd.DataFrame:
    if os.path.exists(path):
        df = pd.read_csv(path, parse_dates=['timestamp'])
        return df.sort_values(['zone_id', 'timestamp']).reset_index(drop=True)
    else:
        return generate_demo_timeseries()

def load_geojson(path: str) -> dict:
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    else:
        return generate_demo_geojson()

def list_zones(df: pd.DataFrame) -> list:
    return sorted(df['zone_id'].unique().tolist())

## 🧪 4. Demo-data generator

Als er geen echte dataset beschikbaar is, maken we een synthetische tijdreeks 
met pseudo-random densities, weersvariabelen en PT-arrivals.

In [ ]:
def generate_demo_timeseries(n_zones: int = 6, days: int = 1, freq="5min") -> pd.DataFrame:
    start = pd.Timestamp(datetime(2025, 8, 20, 10, 0))
    idx = pd.date_range(start, periods=int((24*60)/5*days), freq=freq)
    zone_ids = [f"Z{i+1}" for i in range(n_zones)]

    rows = []
    rng = np.random.default_rng(42)
    for z in zone_ids:
        base = np.clip(np.sin(np.linspace(0, 3*np.pi, len(idx))) * 20 + 40 + rng.normal(0, 5, len(idx)), 0, 100)
        pt = np.abs(np.sin(np.linspace(0, 5*np.pi, len(idx))) * 30 + rng.normal(0, 5, len(idx))).astype(int)
        temp = 20 + 5*np.sin(np.linspace(0, 2*np.pi, len(idx))) + rng.normal(0, 0.5, len(idx))
        wind = 8 + 2*np.cos(np.linspace(0, 2*np.pi, len(idx))) + rng.normal(0, 0.5, len(idx))
        event = (np.sin(np.linspace(0, 6*np.pi, len(idx))) > 0.9).astype(int)
        density = np.clip(base + 0.4*pt + 10*event + rng.normal(0, 3, len(idx)), 0, 200)

        rows.append(pd.DataFrame({
            'timestamp': idx,
            'zone_id': z,
            'density': density,
            'pt_arrivals': pt,
            'temp': temp,
            'wind': wind,
            'special_event': event
        }))
    return pd.concat(rows, ignore_index=True)

## 🌍 5. Demo GeoJSON-generator

In [ ]:
def generate_demo_geojson(n_zones: int = 6) -> dict:
    center_lon, center_lat = 4.914, 52.377
    features = []
    step = 0.01
    k = 0
    for r in range(2):
        for c in range(3):
            k += 1
            z = f"Z{k}"
            lon0 = center_lon + (c-1)*step
            lat0 = center_lat + (r-0.5)*step
            poly = [
                [lon0-0.004, lat0-0.003],
                [lon0+0.004, lat0-0.003],
                [lon0+0.004, lat0+0.003],
                [lon0-0.004, lat0+0.003],
                [lon0-0.004, lat0-0.003]
            ]
            features.append({
                "type": "Feature",
                "properties": {"zone_id": z, "name": f"Zone {z}"},
                "geometry": {"type": "Polygon", "coordinates": [poly]}
            })
            if k >= n_zones:
                break
        if k >= n_zones:
            break
    return {"type": "FeatureCollection", "features": features}

## 🤖 6. Simpele voorspellingsmodellen

We gebruiken drie basisbenaderingen:
1. **Persistence:** voorspelling = laatste gemeten waarde.  
2. **Moving Average:** gemiddelde van de laatste N waarden.  
3. **Random Forest:** eenvoudige ML-benadering met lags + covariaten.

In [ ]:
def forecast_persistence(df_zone, horizon_steps):
    last_val = df_zone['density'].iloc[-1]
    return pd.Series([last_val]*horizon_steps)

def forecast_moving_avg(df_zone, horizon_steps, window=3):
    hist_mean = df_zone['density'].iloc[-window:].mean()
    return pd.Series([hist_mean]*horizon_steps)

def train_rf_and_predict(df_zone, horizon_steps):
    zone = df_zone.copy().reset_index(drop=True)
    for lag in [1, 2, 3]:
        zone[f"density_lag{lag}"] = zone['density'].shift(lag)
    zone = zone.dropna().reset_index(drop=True)

    feats = ['density_lag1', 'density_lag2', 'density_lag3', 'pt_arrivals', 'temp', 'wind', 'special_event']
    X, y = zone[feats], zone['density']

    if len(zone) < 20:
        return forecast_persistence(df_zone, horizon_steps)

    model = RandomForestRegressor(n_estimators=200, random_state=0)
    model.fit(X, y)

    current_feats = zone.iloc[-1:][feats].values.astype(float)
    preds = []
    for _ in range(horizon_steps):
        yhat = model.predict(current_feats)[0]
        preds.append(yhat)
        current_feats[0, :3] = [yhat] + list(current_feats[0, :2])
    return pd.Series(preds)

## 🧱 7. Dashboardlogica (Streamlit)

**Let op:** De onderstaande code is bedoeld voor een `streamlit`-app (bijv. in `app/dashboard.py`), 
en draait niet als interactieve widget in het notebook. Gebruik lokaal:

```bash
streamlit run app/dashboard.py
```

Of test de functies hier met demo-data (zonder UI).

In [ ]:
# Test met demo-data (zonder Streamlit UI):
data_path = "data/crowd_timeseries.csv"
geo_path = "data/zones.geojson"

df = load_timeseries(data_path)
gj = load_geojson(geo_path)

zones = list_zones(df)
zone_sel = zones[0] if zones else None
print("Beschikbare zones:", zones)
print("Geselecteerde zone:", zone_sel)

horizon_min = 15
step_minutes = 5
horizon_steps = horizon_min // step_minutes

df_zone = df[df['zone_id'] == zone_sel].copy()
yhat = forecast_moving_avg(df_zone, horizon_steps)

future_idx = pd.date_range(df_zone['timestamp'].iloc[-1] + timedelta(minutes=step_minutes),
                           periods=horizon_steps, freq=f"{step_minutes}min")
df_forecast = pd.DataFrame({'timestamp': future_idx, 'zone_id': zone_sel, 'density_pred': yhat})

df.head(), df_forecast.head()

## 🧩 8. Samenvatting

✅ **Wat dit dashboard laat zien:**
- Real-time crowd density-voorspellingen per zone.
- Interactieve visualisatie van actuele en voorspelde drukte.
- Waarschuwingen wanneer de voorspelde drukte boven een drempel komt.

📦 **Hoe te gebruiken met Streamlit:**
1. Kopieer de model- en helperfuncties uit dit notebook naar `app/dashboard.py` (of importeer ze).
2. Voeg eigen data toe in `data/crowd_timeseries.csv` (kolommen: `timestamp, zone_id, density, pt_arrivals, temp, wind, special_event`).
3. Voeg een `zones.geojson` toe waarin elke feature `properties.zone_id` bevat.
4. Draai lokaal met:
   ```bash
   streamlit run app/dashboard.py
   ```
5. (Optioneel) Host op Streamlit Cloud via GitHub.